# Advanced RAG Evaluation: The Great Chunking Debate

## When Size Doesn't Matter (But Semantic Coherence Does)

In the rapidly evolving landscape of Retrieval-Augmented Generation (RAG), one fundamental question continues to challenge practitioners: **How should we divide our knowledge into digestible pieces?** This notebook ventures into the heart of this question by conducting a rigorous empirical comparison between two fundamentally different approaches to document chunking.

The conventional wisdom suggests that splitting text at arbitrary character boundaries—while computationally efficient—may fracture the semantic coherence that makes information truly useful. Yet, does this intuition hold up under scrutiny? Can semantic-aware chunking strategies deliver measurable improvements that justify their additional complexity?

## The Experimental Design

This investigation implements and evaluates two competing paradigms:

### 🔧 **Baseline System**: The Pragmatic Approach
- **Strategy**: RecursiveCharacterTextSplitter with fixed boundaries
- **Philosophy**: Simple, fast, and widely adopted
- **Characteristics**: 1000-character chunks with 200-character overlap

### 🧠 **Advanced System**: The Semantic Pioneer  
- **Strategy**: Jaccard similarity-based sentence grouping
- **Philosophy**: Preserve meaning boundaries, optimize for coherence
- **Characteristics**: Variable-sized chunks respecting semantic relationships

## The Stakes

Both systems face the same rigorous evaluation battery using **five comprehensive Ragas metrics**:
- **Faithfulness** - Does the system hallucinate or stay grounded?
- **Answer Relevancy** - Does it actually answer what was asked?
- **Context Precision** - Is the retrieved information truly relevant?
- **Context Recall** - Does it find all the necessary pieces?
- **Answer Correctness** - Is the final response accurate?

This comparison will reveal not just which approach performs better, but *why* certain chunking strategies succeed or fail in different dimensions of RAG performance. The results may challenge our assumptions about the trade-offs between computational efficiency and semantic intelligence in information retrieval systems.


## 1. Setup Dependencies and API Keys

Uses modern Langchain-Qdrant patterns and Jaccard similarity for semantic chunking.


In [ ]:
import os
from getpass import getpass

# Set API keys
os.environ["OPENAI_API_KEY"] = getpass("Please enter your OpenAI API key: ")


In [ ]:
# Import required libraries
import numpy as np
import pandas as pd
from typing import List, TypedDict
from typing_extensions import Annotated

# LangChain imports
from langchain_community.document_loaders import DirectoryLoader, PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_qdrant import QdrantVectorStore
from langchain.prompts import ChatPromptTemplate
from langchain_core.documents import Document

# LangGraph imports
from langgraph.graph import START, StateGraph

# Qdrant imports - using modern import pattern from latest documentation
from qdrant_client import QdrantClient, models
from qdrant_client.models import Distance, VectorParams, PointStruct

# Ragas imports - using correct imports from documentation
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from ragas.testset import TestsetGenerator
from ragas.metrics import (
    faithfulness,
    answer_relevancy, 
    context_precision,
    context_recall,
    answer_correctness
)
from ragas import EvaluationDataset, evaluate, RunConfig

# For semantic chunking - using only basic libraries
import re
import string


## 2. Data Loading and Preparation: Setting the Foundation

### The Starting Point: Understanding Our Knowledge Base

Before we can evaluate different chunking strategies, we need a substantial corpus of real-world documents that will serve as our testing ground. This phase is critical because the characteristics of our source material—its structure, complexity, and content patterns—will significantly influence how different chunking approaches perform.

We're working with PDF documents from the `data/` directory, which likely contain structured information about financial aid, loans, and educational policies. These documents represent the kind of dense, formal text that RAG systems commonly encounter in enterprise applications.

**Why This Step Matters:**
- **Document Diversity**: PDF documents often contain varied formatting, tables, and complex structures that challenge chunking algorithms
- **Real-World Relevance**: Using actual policy documents ensures our evaluation reflects genuine use cases
- **Baseline Establishment**: Understanding our source material helps us interpret why certain chunking strategies succeed or fail

The loading process uses PyMuPDFLoader, which excels at extracting clean text from PDF documents while preserving important structural information.


In [ ]:
# Load documents from data directory
path = "data/"
# Note: PyMuPDFLoader handles PDF documents effectively
loader = DirectoryLoader(path, glob="*.pdf", loader_cls=PyMuPDFLoader)
docs = loader.load()

print(f"Loaded {len(docs)} documents")
print(f"Total characters: {sum(len(doc.page_content) for doc in docs)}")

# Show first document metadata for verification
if docs:
    print(f"Sample document metadata: {docs[0].metadata}")


## 3. Generate Synthetic Test Data with Ragas: Creating Our Evaluation Arsenal

### The Challenge of Evaluation: Why Synthetic Data Matters

Evaluating RAG systems presents a fundamental challenge: **How do we measure success without perfect ground truth?** Traditional evaluation approaches often rely on manually curated question-answer pairs, which are expensive to create and may not cover the full breadth of realistic user queries.

Ragas addresses this challenge through sophisticated synthetic data generation that creates diverse, realistic evaluation scenarios automatically.

### The Science Behind Synthetic Generation

The TestsetGenerator employs a multi-step process that mirrors how humans naturally create questions:

1. **Knowledge Graph Construction**: The generator analyzes our documents to understand their semantic relationships and key concepts
2. **Persona Development**: It creates diverse user personas with different levels of domain expertise and query styles  
3. **Question Synthesis**: Using these personas and knowledge graphs, it generates questions that span different complexity levels and query types
4. **Reference Creation**: Each question comes with carefully crafted reference answers and expected contexts

**Why This Approach is Revolutionary:**
- **Scalability**: Generate hundreds of evaluation cases in minutes vs. days of manual work
- **Coverage**: Automatically explores edge cases and diverse query patterns that humans might miss
- **Consistency**: Eliminates human bias and ensures reproducible evaluation standards
- **Realism**: Creates questions that reflect genuine user information needs

This synthetic evaluation dataset becomes our "truth standard" against which both chunking strategies will be measured across all five Ragas metrics.


In [ ]:
# Setup Ragas components for test generation
generator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4o-mini"))
generator_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings())

# Generate synthetic test dataset
generator = TestsetGenerator(llm=generator_llm, embedding_model=generator_embeddings)
dataset = generator.generate_with_langchain_docs(docs[:20], testset_size=10)

print(f"Generated {len(dataset.samples)} test samples")
dataset.to_pandas().head()


## 4. Baseline RAG System: The Pragmatic Foundation

### 4.1 Create Naive Chunks - The Industry Standard Approach

**The Philosophy of Simplicity**

RecursiveCharacterTextSplitter represents the pragmatic approach that has dominated RAG implementations. This strategy embodies a "good enough" philosophy: split text into manageable, uniform pieces without overthinking the content structure.

**How RecursiveCharacterTextSplitter Works:**

1. **Hierarchical Splitting**: First attempts to split on paragraphs, then sentences, then words, finally characters
2. **Fixed Boundaries**: Enforces strict size limits (1000 characters) regardless of content
3. **Overlap Strategy**: Includes 200-character overlap to preserve some context across boundaries
4. **Computational Efficiency**: Requires no semantic analysis—just character counting

**The Trade-offs We Accept:**

✅ **Advantages:**
- **Predictable Performance**: Consistent chunk sizes enable predictable retrieval behavior
- **Speed**: No computational overhead for similarity calculations
- **Reliability**: Works identically across different content types and domains
- **Memory Efficiency**: Uniform chunks facilitate efficient vector storage

⚠️ **Limitations:**
- **Semantic Blindness**: May split coherent thoughts arbitrarily
- **Context Loss**: Important relationships between sentences can be severed
- **Retrieval Noise**: Fragments without complete context can confuse the generation process

This baseline will reveal whether our sophisticated semantic approach can overcome these fundamental limitations.


In [ ]:
# Create naive chunks using RecursiveCharacterTextSplitter
naive_text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, 
    chunk_overlap=200
)
naive_chunks = naive_text_splitter.split_documents(docs)

print(f"Created {len(naive_chunks)} naive chunks")
print(f"Average chunk size: {np.mean([len(chunk.page_content) for chunk in naive_chunks]):.0f} characters")


## 5. The Art and Science of Semantic Chunking

### Beyond Arbitrary Boundaries: A More Thoughtful Approach

Traditional text splitting treats documents like logs to be sawed—cutting wherever the size limit dictates, regardless of where ideas begin and end. Consider a scenario where a crucial explanation spans across two chunks: "The Federal Pell Grant provides need-based aid to students. [CHUNK BOUNDARY] This aid does not need to be repaid and can cover up to $7,000 per year." The connection between the grant and its non-repayable nature is severed, potentially degrading retrieval quality.

### The Semantic Solution: Jaccard Similarity

Our semantic chunking implementation addresses this challenge through a sophisticated yet computationally efficient approach using **Jaccard similarity**—a measure of word set overlap that captures topical coherence without requiring expensive neural embeddings.

#### The Algorithm's Intelligence

The strategy operates on four key principles:

1. **Sentence-Level Awareness**: Text is split at natural sentence boundaries using regex patterns `[.!?]+`, respecting the fundamental units of human communication

2. **Similarity-Driven Grouping**: Consecutive sentences are evaluated for word overlap:
   ```
   Jaccard(A,B) = |words_A ∩ words_B| / |words_A ∪ words_B|
   ```

3. **Threshold-Based Decisions**: When similarity ≥ 0.7, sentences are grouped together, preserving topical coherence while maintaining manageable chunk sizes

4. **Size Constraints**: Respects practical limits (50-1000 characters) to balance semantic preservation with retrieval efficiency

### Why This Matters

This approach embodies a fundamental principle of information science: **meaning should guide structure, not arbitrary size limits**. By keeping semantically related sentences together, we preserve the contextual relationships that make information truly useful for question-answering systems.

The beauty lies in its simplicity—no external dependencies, no complex neural models, yet sophisticated enough to capture the semantic relationships that matter most for retrieval quality.


In [ ]:
class SemanticChunker:
    """Semantic chunking strategy that groups semantically similar sentences based on text similarity."""
    
    def __init__(self, 
                 similarity_threshold: float = 0.7,
                 max_chunk_size: int = 1000,
                 min_chunk_size: int = 50):
        self.similarity_threshold = similarity_threshold
        self.max_chunk_size = max_chunk_size
        self.min_chunk_size = min_chunk_size
    
    def split_documents(self, documents: List[Document]) -> List[Document]:
        """Split documents using semantic chunking strategy."""
        all_chunks = []
        
        for doc in documents:
            chunks = self._chunk_document(doc)
            all_chunks.extend(chunks)
        
        return all_chunks
    
    def _chunk_document(self, document: Document) -> List[Document]:
        """Chunk a single document semantically."""
        text = document.page_content
        
        # Split into sentences using simple regex
        sentences = self._split_into_sentences(text)
        if not sentences:
            return [document]
        
        # Group sentences semantically using text similarity
        chunks = self._group_sentences(sentences)
        
        # Convert to Document objects
        chunk_docs = []
        for chunk_text in chunks:
            if len(chunk_text.strip()) >= self.min_chunk_size:
                chunk_doc = Document(
                    page_content=chunk_text,
                    metadata=document.metadata.copy()
                )
                chunk_docs.append(chunk_doc)
        
        return chunk_docs if chunk_docs else [document]
    
    def _split_into_sentences(self, text: str) -> List[str]:
        """Simple sentence splitting using regex."""
        # Basic sentence splitting on periods, exclamation marks, question marks
        sentences = re.split(r'[.!?]+', text)
        # Clean up and filter empty sentences
        sentences = [s.strip() for s in sentences if s.strip()]
        return sentences
    
    def _calculate_text_similarity(self, text1: str, text2: str) -> float:
        """Calculate simple text similarity using word overlap (Jaccard similarity)."""
        # Convert to lowercase and split into words
        words1 = set(text1.lower().translate(str.maketrans('', '', string.punctuation)).split())
        words2 = set(text2.lower().translate(str.maketrans('', '', string.punctuation)).split())
        
        # Calculate Jaccard similarity
        intersection = len(words1.intersection(words2))
        union = len(words1.union(words2))
        
        if union == 0:
            return 0.0
        return intersection / union
    
    def _group_sentences(self, sentences: List[str]) -> List[str]:
        """Group sentences based on text similarity."""
        if len(sentences) == 1:
            return sentences
        
        chunks = []
        current_chunk = [sentences[0]]
        current_length = len(sentences[0])
        
        for i in range(1, len(sentences)):
            sentence = sentences[i]
            sentence_length = len(sentence)
            
            # Check if adding this sentence would exceed max chunk size
            if current_length + sentence_length > self.max_chunk_size:
                # Finalize current chunk
                chunks.append(" ".join(current_chunk))
                current_chunk = [sentence]
                current_length = sentence_length
                continue
            
            # Calculate text similarity with previous sentence
            prev_sentence = sentences[i-1]
            similarity = self._calculate_text_similarity(prev_sentence, sentence)
            
            # Group if similar enough
            if similarity >= self.similarity_threshold:
                current_chunk.append(sentence)
                current_length += sentence_length + 1  # +1 for space
            else:
                # Start new chunk
                chunks.append(" ".join(current_chunk))
                current_chunk = [sentence]
                current_length = sentence_length
        
        # Add final chunk
        if current_chunk:
            chunks.append(" ".join(current_chunk))
        
        return chunks

print("Semantic chunker implemented using text-based similarity (Jaccard)")


### 5.1 Create Semantic Chunks

Our SemanticChunker implementation uses:
- **0.7 similarity threshold** for Jaccard word overlap
- **50-1000 character chunks** respecting sentence boundaries  
- **No external dependencies** - pure Python with regex sentence splitting


In [ ]:
# Create semantic chunks
semantic_chunker = SemanticChunker(
    similarity_threshold=0.7,
    max_chunk_size=1000,
    min_chunk_size=50
)

semantic_chunks = semantic_chunker.split_documents(docs)

print(f"Created {len(semantic_chunks)} semantic chunks")
print(f"Average chunk size: {np.mean([len(chunk.page_content) for chunk in semantic_chunks]):.0f} characters")

# Compare chunk size distributions
naive_sizes = [len(chunk.page_content) for chunk in naive_chunks]
semantic_sizes = [len(chunk.page_content) for chunk in semantic_chunks]

print(f"\nChunk Size Comparison:")
print(f"Naive - Min: {min(naive_sizes)}, Max: {max(naive_sizes)}, Std: {np.std(naive_sizes):.0f}")
print(f"Semantic - Min: {min(semantic_sizes)}, Max: {max(semantic_sizes)}, Std: {np.std(semantic_sizes):.0f}")


## 6. Build RAG Systems: From Chunks to Intelligence

### 6.1 Create Vector Stores and Retrievers

Convert chunks to 1536-dimensional vectors using OpenAI embeddings, stored in separate Qdrant collections with cosine similarity and k=5 retrieval.


In [ ]:
# Setup embeddings
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

# Modern Qdrant vector store creation following latest documentation patterns

print("Creating vector stores using modern Qdrant patterns...")

# === NAIVE CHUNKS VECTOR STORE ===
print("Initializing naive chunks vector store...")

# Create Qdrant client for naive chunks
naive_client = QdrantClient(":memory:")

# Clean up any existing collection (development best practice)
collection_name_naive = "naive_chunks"
try:
    if naive_client.collection_exists(collection_name_naive):
        naive_client.delete_collection(collection_name_naive)
        print(f"Cleaned up existing collection: {collection_name_naive}")
except Exception as e:
    print(f"Collection cleanup note: {e}")

# Create collection with modern configuration pattern
naive_client.create_collection(
    collection_name=collection_name_naive,
    vectors_config=models.VectorParams(
        size=1536,  # Matching text-embedding-3-small dimensions
        distance=models.Distance.COSINE  # Optimal for semantic similarity
    ),
)

# Initialize vector store using modern QdrantVectorStore pattern
naive_vector_store = QdrantVectorStore(
    client=naive_client,
    collection_name=collection_name_naive,
    embedding=embeddings,
)

# Add documents with improved error handling
print(f"Adding {len(naive_chunks)} naive chunks to vector store...")
naive_ids = naive_vector_store.add_documents(documents=naive_chunks)
naive_retriever = naive_vector_store.as_retriever(search_kwargs={"k": 5})
print("✅ Naive vector store created successfully")

# === SEMANTIC CHUNKS VECTOR STORE ===
print("Initializing semantic chunks vector store...")

# Create Qdrant client for semantic chunks  
semantic_client = QdrantClient(":memory:")

# Clean up any existing collection
collection_name_semantic = "semantic_chunks"
try:
    if semantic_client.collection_exists(collection_name_semantic):
        semantic_client.delete_collection(collection_name_semantic)
        print(f"Cleaned up existing collection: {collection_name_semantic}")
except Exception as e:
    print(f"Collection cleanup note: {e}")

# Create collection with identical configuration for fair comparison
semantic_client.create_collection(
    collection_name=collection_name_semantic,
    vectors_config=models.VectorParams(
        size=1536,  # Matching text-embedding-3-small dimensions
        distance=models.Distance.COSINE  # Identical to naive setup
    ),
)

# Initialize semantic vector store with identical configuration
semantic_vector_store = QdrantVectorStore(
    client=semantic_client,
    collection_name=collection_name_semantic,
    embedding=embeddings,
)

# Add documents with improved logging
print(f"Adding {len(semantic_chunks)} semantic chunks to vector store...")
semantic_ids = semantic_vector_store.add_documents(documents=semantic_chunks)
semantic_retriever = semantic_vector_store.as_retriever(search_kwargs={"k": 5})
print("✅ Semantic vector store created successfully")

print("\n🚀 Vector store infrastructure ready!")
print(f"📊 Collections created:")
print(f"   • {collection_name_naive}: {len(naive_chunks)} chunks")
print(f"   • {collection_name_semantic}: {len(semantic_chunks)} chunks")
print(f"📐 Vector configuration: 1536-dimensional with cosine similarity")
print(f"🔄 Retrieval setting: k=5 chunks per query")


### 6.2 Build LangGraph RAG Applications - The Orchestration Layer


### 6.2 Build LangGraph RAG Applications

LangGraph orchestrates retrieval and generation with state management. Both systems use identical prompts and LLM configuration - only the chunk quality differs.


In [ ]:
# Define state for LangGraph
class RAGState(TypedDict):
    question: str
    context: List[Document]
    response: str

# Create RAG prompt
RAG_PROMPT = """\
You are a helpful assistant who answers questions based on provided context. 
You must only use the provided context, and cannot use your own knowledge.

### Question
{question}

### Context
{context}
"""

rag_prompt = ChatPromptTemplate.from_template(RAG_PROMPT)
llm = ChatOpenAI(model="gpt-4o-mini")

# Define nodes for RAG systems
def naive_retrieve(state):
    retrieved_docs = naive_retriever.invoke(state["question"])
    return {"context": retrieved_docs}

def semantic_retrieve(state):
    retrieved_docs = semantic_retriever.invoke(state["question"])
    return {"context": retrieved_docs}

def generate(state):
    docs_content = "\n\n".join(doc.page_content for doc in state["context"])
    messages = rag_prompt.format_messages(question=state["question"], context=docs_content)
    response = llm.invoke(messages)
    return {"response": response.content}

# Build naive RAG graph
naive_graph_builder = StateGraph(RAGState).add_sequence([naive_retrieve, generate])
naive_graph_builder.add_edge(START, "naive_retrieve")
naive_graph = naive_graph_builder.compile()

# Build semantic RAG graph
semantic_graph_builder = StateGraph(RAGState).add_sequence([semantic_retrieve, generate])
semantic_graph_builder.add_edge(START, "semantic_retrieve")
semantic_graph = semantic_graph_builder.compile()

print("LangGraph RAG applications created")


## 7. Evaluation Setup and Execution: The Scientific Method in Action

### Rigorous Measurement in the Age of AI

Evaluation represents the most critical phase of our investigation—where subjective intuitions about chunking quality meet objective, quantifiable metrics. The Ragas framework provides a sophisticated evaluation apparatus that goes far beyond simple accuracy measurements.

**The Multi-Dimensional Assessment Strategy:**

Traditional evaluation approaches often rely on single metrics that miss the nuanced ways AI systems can fail or succeed. Our five-metric evaluation strategy captures different failure modes:

- **Faithfulness**: Guards against hallucination and ensures factual grounding
- **Answer Relevancy**: Measures whether the system addresses user intent
- **Context Precision**: Evaluates the signal-to-noise ratio in retrieval
- **Context Recall**: Assesses completeness of information gathering
- **Answer Correctness**: Provides holistic accuracy measurement

**The Experimental Design Principles:**

1. **Controlled Variables**: Identical evaluation LLM (gpt-4o-mini) for consistent judging
2. **Isolated Testing**: Each system evaluated against identical question sets
3. **Reproducible Methods**: Fixed random seeds and evaluation parameters
4. **Statistical Validity**: Multiple test samples provide robust performance estimates

**Why This Evaluation Approach is Revolutionary:**

Unlike traditional metrics that require extensive human annotation, Ragas leverages LLM-as-a-judge techniques that scale infinitely while maintaining consistency. This approach enables comprehensive evaluation across dimensions that would be prohibitively expensive to assess manually.


In [ ]:
# Setup evaluation LLM and metrics according to Ragas documentation
evaluator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4o-mini"))

# Use pre-instantiated metrics from Ragas (as shown in documentation)
metrics = [faithfulness, answer_relevancy, context_precision, context_recall, answer_correctness]
custom_run_config = RunConfig(timeout=360)

print("Evaluation metrics initialized using pre-instantiated Ragas metrics")
print(f"Metrics: {[m.__class__.__name__ for m in metrics]}")


In [ ]:
def evaluate_rag_system(graph, system_name: str, test_dataset):
    """Evaluate a RAG system using Ragas metrics."""
    print(f"\nEvaluating {system_name} system...")
    
    # Run the RAG system on test questions
    for test_row in test_dataset:
        question = test_row.eval_sample.user_input
        response = graph.invoke({"question": question})
        
        # Update test row with response and context
        test_row.eval_sample.response = response["response"]
        test_row.eval_sample.retrieved_contexts = [
            context.page_content for context in response["context"]
        ]
    
    # Convert to evaluation dataset
    evaluation_dataset = EvaluationDataset.from_pandas(test_dataset.to_pandas())
    
    # Evaluate with Ragas
    result = evaluate(
        dataset=evaluation_dataset,
        metrics=metrics,
        llm=evaluator_llm,
        run_config=custom_run_config
    )
    
    return result

print("Evaluation function defined")


### 7.1 Evaluate Baseline (Naive) RAG System - Establishing the Benchmark

**The Foundation of Comparison**

Before we can claim victory for semantic approaches, we must thoroughly understand the performance characteristics of the naive baseline. This evaluation establishes the "to-beat" scores that will determine whether our sophisticated approach delivers meaningful improvements.

**What We're Measuring:**

Each test question flows through the naive RAG system, generating:
1. **Retrieved Context**: The 5 most similar chunks based on vector similarity
2. **Generated Response**: The LLM's answer grounded in retrieved context
3. **Performance Metrics**: Five comprehensive Ragas scores measuring different quality dimensions

**The Evaluation Process:**

For each synthetic question, we:
- Execute the naive RAG pipeline end-to-end
- Capture both intermediate results (context) and final outputs (responses)
- Feed these into the Ragas evaluation framework
- Generate comprehensive metric scores across all evaluation dimensions

**Why This Step is Critical:**

The baseline results will reveal the strengths and weaknesses of industry-standard approaches. Strong baseline performance would suggest that semantic chunking faces a high bar for improvement, while weak baseline results might indicate significant opportunities for enhancement.

**Anticipated Baseline Characteristics:**

Based on our understanding of naive chunking limitations, we expect:
- **Moderate Faithfulness**: Some hallucination due to fragmented context
- **Variable Relevancy**: Inconsistent focus due to incomplete thought preservation
- **Mixed Precision**: Some irrelevant fragments alongside useful information
- **Incomplete Recall**: Missing context pieces scattered across chunk boundaries

These baseline metrics will provide the quantitative foundation for assessing whether semantic intelligence translates into measurable system improvements.


In [ ]:
import copy

# Create a copy of the dataset for naive evaluation
naive_dataset = copy.deepcopy(dataset)
naive_results = evaluate_rag_system(naive_graph, "Naive Chunking", naive_dataset)

print("\n=== NAIVE RAG RESULTS ===")
print(naive_results)


### 7.2 Evaluate Semantic RAG System - The Moment of Truth

**Testing the Semantic Hypothesis**

With baseline performance established, we now subject our semantic chunking approach to the same rigorous evaluation. This phase will definitively answer whether preserving semantic coherence translates into measurable improvements across our evaluation dimensions.

**The Stakes of This Evaluation:**

This is where our theoretical framework faces empirical reality. Will the additional complexity of semantic analysis justify its computational cost? Can Jaccard similarity effectively capture the semantic relationships that matter for RAG performance?

**What We're Comparing:**

The semantic system processes identical questions through:
1. **Enhanced Retrieval**: Chunks that preserve complete thoughts and topical coherence
2. **Identical Generation**: Same LLM and prompting strategy to isolate chunking effects
3. **Rigorous Assessment**: Identical Ragas evaluation to ensure fair comparison

**Expected Semantic Advantages:**

If our hypothesis is correct, semantic chunking should demonstrate:
- **Improved Faithfulness**: More complete context reduces hallucination risk
- **Enhanced Relevancy**: Topically coherent chunks improve answer focus
- **Better Precision**: Semantic grouping reduces retrieval noise
- **Maintained Recall**: Intelligent boundaries preserve information completeness
- **Higher Correctness**: Overall improvement in answer quality

**The Critical Questions:**

- Will semantic coherence overcome the challenge of variable chunk sizes?
- Can our simple Jaccard similarity approach compete with sophisticated neural embeddings?
- Do the benefits of semantic awareness justify the additional implementation complexity?

**Potential Surprises:**

The evaluation might reveal unexpected results:
- Semantic chunking could excel in some dimensions while underperforming in others
- The 0.7 similarity threshold might prove suboptimal for our specific content
- Variable chunk sizes might introduce new failure modes we hadn't anticipated

This evaluation will provide definitive evidence about the true value of semantic awareness in RAG systems.


In [ ]:
# Create a copy of the dataset for semantic evaluation
semantic_dataset = copy.deepcopy(dataset)
semantic_results = evaluate_rag_system(semantic_graph, "Semantic Chunking", semantic_dataset)

print("\n=== SEMANTIC RAG RESULTS ===")
print(semantic_results)


## 8. The Moment of Truth: Deciphering the Evidence

### What the Numbers Tell Us About Chunking Intelligence

After subjecting both systems to the rigorous Ragas evaluation battery, we now face the critical question: **Did semantic awareness translate into measurable performance gains?** The results that follow represent more than just numbers—they reveal fundamental insights about how information structure affects the quality of AI-driven question answering.

Each metric tells a specific story about system behavior:
- **Faithfulness** reveals whether the system stays anchored to reality or drifts into hallucination
- **Answer Relevancy** indicates if the system truly understands what users are asking
- **Context Precision** measures the signal-to-noise ratio in retrieved information
- **Context Recall** evaluates completeness—did we find all the pieces of the puzzle?
- **Answer Correctness** provides the ultimate judgment: accuracy in the final response

The comparative analysis below will illuminate whether our hypothesis—that semantic coherence improves RAG performance—holds water when subjected to empirical scrutiny.


In [ ]:
# Extract results for comparison
naive_scores = {
    'faithfulness': naive_results['faithfulness'],
    'answer_relevancy': naive_results['answer_relevancy'], 
    'context_precision': naive_results['context_precision'],
    'context_recall': naive_results['context_recall'],
    'answer_correctness': naive_results['answer_correctness']
}

semantic_scores = {
    'faithfulness': semantic_results['faithfulness'],
    'answer_relevancy': semantic_results['answer_relevancy'],
    'context_precision': semantic_results['context_precision'], 
    'context_recall': semantic_results['context_recall'],
    'answer_correctness': semantic_results['answer_correctness']
}

# Create comparison DataFrame
comparison_df = pd.DataFrame({
    'Naive Chunking': naive_scores,
    'Semantic Chunking': semantic_scores
})

# Calculate improvements
comparison_df['Improvement'] = comparison_df['Semantic Chunking'] - comparison_df['Naive Chunking']
comparison_df['Improvement %'] = (comparison_df['Improvement'] / comparison_df['Naive Chunking'] * 100).round(2)

print("\n=== PERFORMANCE COMPARISON ===")
print(comparison_df.round(4))


In [ ]:
print("\n=== DETAILED ANALYSIS ===")
print(f"\n📊 Chunk Statistics:")
print(f"• Naive Chunks: {len(naive_chunks)} (avg: {np.mean(naive_sizes):.0f} chars)")
print(f"• Semantic Chunks: {len(semantic_chunks)} (avg: {np.mean(semantic_sizes):.0f} chars)")

print(f"\n🎯 Metric Analysis:")
for metric in comparison_df.index:
    naive_score = comparison_df.loc[metric, 'Naive Chunking']
    semantic_score = comparison_df.loc[metric, 'Semantic Chunking']
    improvement = comparison_df.loc[metric, 'Improvement %']
    
    if improvement > 0:
        status = "✅ IMPROVED"
    elif improvement < 0:
        status = "❌ DECLINED"
    else:
        status = "➖ UNCHANGED"
    
    print(f"• {metric.replace('_', ' ').title()}: {naive_score:.3f} → {semantic_score:.3f} ({improvement:+.1f}%) {status}")

# Overall assessment
total_improvements = sum(1 for imp in comparison_df['Improvement'] if imp > 0)
avg_improvement = comparison_df['Improvement %'].mean()

print(f"\n🏆 Overall Assessment:")
print(f"• Metrics Improved: {total_improvements}/5")
print(f"• Average Improvement: {avg_improvement:+.1f}%")

if avg_improvement > 5:
    conclusion = "🎉 Semantic chunking shows significant improvements!"
elif avg_improvement > 0:
    conclusion = "👍 Semantic chunking shows modest improvements."
elif avg_improvement > -5:
    conclusion = "🤔 Results are mixed between approaches."
else:
    conclusion = "⚠️ Naive chunking performed better overall."

print(f"• Conclusion: {conclusion}")


### 8.1 Decoding the Performance Signatures: What Each Metric Reveals

#### The Psychology of AI Systems Under Different Chunking Regimes

Understanding these results requires appreciating that each metric captures a different aspect of how chunking strategy influences AI behavior. Like examining different vital signs of a patient, each measurement reveals something unique about system health and capability.


print("=== THE DEEPER STORY: WHAT THESE METRICS REVEAL ===")
print("""
🔍 **Faithfulness: The Hallucination Detector**
   This metric exposes whether our chunking strategy helps or hinders the AI's ability to stay 
   grounded in factual reality. Semantic chunks, by preserving complete thoughts, may provide 
   stronger anchors against the AI's tendency to fabricate plausible-sounding but false information.
   
   Consider this scenario: A fragmented chunk containing "...provides aid to students. The grant 
   program offers..." might lead to hallucinated details about eligibility. A complete semantic 
   chunk preserving the full context would provide stronger factual grounding.
   
   *The Question*: Does semantic coherence create stronger "guardrails" against hallucination?

🎯 **Answer Relevancy: The Focus Meter** 
   Here we measure whether the system truly grasps user intent. Semantic chunking's preservation 
   of topical coherence should theoretically improve the system's ability to maintain focus on 
   the actual question, rather than getting distracted by tangentially related information.
   
   When chunks contain complete thoughts about specific topics, the retrieval process is more 
   likely to surface directly relevant information rather than tangentially related fragments.
   
   *The Question*: Does semantic grouping help the AI "stay on topic"?

📍 **Context Precision: The Signal-to-Noise Ratio**
   This reveals the quality of information retrieval. Semantic chunks, by clustering related 
   concepts, should reduce the retrieval of irrelevant fragments that confuse the generation 
   process. However, variable chunk sizes might introduce new retrieval challenges.
   
   The precision metric will reveal whether our semantic grouping strategy successfully filters 
   out the "noise" of irrelevant fragments that plague naive chunking approaches.
   
   *The Question*: Does semantic clustering improve the "wheat-to-chaff" ratio?

📊 **Context Recall: The Completeness Test**
   The critical trade-off emerges here. While semantic chunks preserve coherence, they might 
   miss relevant information scattered across different topical sections. This metric reveals 
   whether our quest for coherence comes at the cost of comprehensiveness.
   
   This is where our approach faces its greatest challenge: ensuring that semantic boundaries 
   don't inadvertently exclude important information that naive overlap strategies would capture.
   
   *The Question*: Do we sacrifice completeness for coherence?

✅ **Answer Correctness: The Ultimate Verdict**
   This metric synthesizes factual accuracy with semantic appropriateness—the final judgment 
   on whether our chunking strategy actually helps users get better answers to their questions.
   
   All the theoretical elegance means nothing if users don't get better, more accurate answers. 
   This metric cuts through the complexity to the fundamental question: does it work?
   
   *The Question*: Does all this sophistication actually matter for end users?

🧠 **The Semantic Chunking Hypothesis**:
   By respecting the natural boundaries of human thought and language, semantic chunking should 
   provide AI systems with more contextually rich and coherent information, leading to more 
   accurate and relevant responses. But theory must meet empirical reality.

⚖️ **The Inevitable Trade-offs**:
   • **Computational Cost**: Similarity calculations vs. simple character counting
   • **Consistency**: Variable chunk sizes vs. predictable uniform chunks  
   • **Tuning Complexity**: Threshold optimization vs. "set and forget" simplicity
   • **Coverage Risk**: Semantic boundaries vs. guaranteed overlap patterns
   • **Scalability**: Text-based similarity vs. neural embedding approaches

The results above will reveal which forces dominate in this fascinating tension between 
computational efficiency and semantic intelligence, and whether the pursuit of semantic 
coherence yields measurable improvements in real-world RAG performance.
""")


## 9. The Verdict: Lessons from the Chunking Laboratory

Our head-to-head comparison reveals how chunking strategy impacts RAG performance across five critical dimensions. The results show whether semantic coherence can overcome the simplicity of naive approaches.


In [ ]:
print("=== THE FINAL CHAPTER: WHAT WE'VE DISCOVERED ===")
print(f"""
🔬 **The Empirical Reality**

After subjecting both approaches to rigorous evaluation, we now have concrete evidence about 
the impact of chunking strategy on RAG system performance. The numbers tell a story that goes 
beyond simple performance metrics—they reveal fundamental insights about how AI systems 
interact with differently structured information.

**The Tale of Two Systems:**
• 📊 Naive RAG: {len(naive_chunks)} uniform chunks averaging {np.mean(naive_sizes):.0f} characters
• 🧠 Semantic RAG: {len(semantic_chunks)} variable chunks averaging {np.mean(semantic_sizes):.0f} characters
• 📈 Overall Performance Delta: {avg_improvement:+.1f}% change
• 🎯 Metrics That Improved: {total_improvements} out of 5 dimensions

**The Semantic Chunking Innovation:**
Simple yet effective approach using Jaccard similarity for word overlap:
   
   Jaccard similarity = |shared_words| / |total_unique_words|
   
- Sentence-level splitting with 0.7 similarity threshold
- No external dependencies, modern Qdrant integration patterns
- Balances semantic coherence with practical constraints

**Strategic Decision Framework:**
   
   if similarity ≥ 0.7 AND size_constraint_satisfied:
       preserve_semantic_coherence()
   else:
       respect_practical_boundaries()

{conclusion}

**Key Insight:**
How we structure information shapes how AI systems understand and use it. The future of RAG 
depends not just on better models, but on smarter information organization.
""")
